Так как это демка, мы решили сделать фиксированный датасет вакансий для подбора. 

Тогда для ускорения работы приложения можно заранее обработать датасет вакансий и получить эмбеддинги содержаний, чтобы потом сравнивать их с резюме.

In [ ]:
from gensim.models.doc2vec import Doc2Vec
import pandas as pd
import numpy as np
import re

import faiss
import PyPDF2

from io import StringIO
from html.parser import HTMLParser

In [ ]:
# HTML stripping (https://stackoverflow.com/questions/753052/strip-html-from-strings-in-python)
class MLStripper(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.text = StringIO()
    def handle_data(self, d):
        self.text.write(d)
    def get_data(self):
        return self.text.getvalue()

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

load dataset

In [ ]:
jobs = pd.read_csv('data/hhparser_vacancy.csv')
jobs.head()

process data

In [ ]:
jobs['content'] = jobs['name'] + ' ' + jobs['description']
jobs['content'] = jobs['content'].astype(str)
jobs['content'] = jobs['content'].map(lambda x: strip_tags(x.lower()))  # Remove HTML, lower case
jobs.head()

get vectors

In [ ]:
model = Doc2Vec.load('model/doc2vec_max.model')

In [ ]:
jobs['embd'] = jobs['content'].apply(lambda x: model.infer_vector(x.split()))
jobs.head()

write vectors to FAISS index and store them. id's should match the jobs csv

In [ ]:
index = faiss.IndexFlatL2(50)  # size from model params in train_d2v
print(index.is_trained)
index.add(np.array(jobs.embd.values.tolist()))
print(index.ntotal)

In [ ]:
faiss.write_index(index, "model/hh.index")

# Testing

In [ ]:
# Fixed dataset of open positions
jobs = pd.read_csv('data/hhparser_vacancy.csv')

# Embeddings
index = faiss.read_index("model/hh.index")

In [ ]:
resume = []

with open('data/sample_resume.pdf', 'rb') as f:
    pdf = PyPDF2.PdfReader(f)
    for page in pdf.pages:
        text = page.extract_text()
        resume.append(text)

# resume = ' '.join([e.replace(' ', '').replace('\n', ' ').lower() for e in resume])
resume = ' '.join([e.replace('\n', ' ').lower() for e in resume])
resume = re.sub(r'[^\w\s]','',resume).replace('\n', ' ')

resume

In [ ]:
# get doc vector
v1 = np.array([model.infer_vector(resume.split())])

In [ ]:
# find the closest embedding in index
distances, indices = index.search(v1, 5)
distances, indices

In [ ]:
jobs.loc[indices[0][0]]